In [1]:
import os
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
# Load dataset
dfpath = os.path.join(os.getcwd(), "data", "training_data.csv")
print("Loading file from: {}".format(dfpath))

# Only store first 999999 rows for testing, comparable to data subset
df = pd.read_csv(dfpath, header=0, nrows=999999)
df.head()

Loading file from: C:\Programming\CustomerAnalysis\GroupAssignment\data\training_data.csv


,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
2,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
3,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
4,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...


In [3]:
# Sessionization
# Taken from practical session file: Assumed this is about as efficient as it gets
df['product_action'] = df['product_action'].fillna('view')
df = df.groupby('session_id_hash')['product_action'].agg(tuple).reset_index()
del df['session_id_hash']
df.shape

(138074, 1)

In [4]:
# Drop all sessions not containing any 'add' events
df.drop(df[df.product_action.map(lambda x: 'add' not in x)].index, inplace=True)
df.shape

(5626, 1)

In [5]:
# Slice the head of the sessions until after the first add event
df['product_action'] = df['product_action'].map(lambda x: x[x.index('add') + 1:-1])
df.head()

,product_action
0,"(view, detail, view, view, view, detail, view,..."
1,"(view, view, view, view, view, view, view, vie..."
10,"(view, remove, view, view, view)"
17,"(remove, view, view, view, detail, view, view,..."
39,"(add, view, remove, view, view, view, view, de..."


In [6]:
# Class labeling
# From practical session.
df['purchase'] = np.where(df.product_action.map(set(['purchase']).issubset), 1, 0)

In [7]:
# Cutting down purchase sessions
# Based on Carlijn Jurriaan's solution on Canvas discussion board
df['product_action'] = df['product_action'].map(lambda x: x[0:x.index('purchase')] if 'purchase' in x else x)

# Clear any empty lists from the database. Not sure if this is strictly necessary, given other filters.
df = df[df['product_action'].str.len() > 0]
df.head()

,product_action,purchase
0,"(view, detail, view, view, view, detail, view,...",0
1,"(view, view, view, view, view, view, view, vie...",0
10,"(view, remove, view, view, view)",0
17,"(remove, view, view, view, detail, view, view,...",0
39,"(add, view, remove, view, view, view, view, de...",1


In [8]:
# Drop all sessions with a length shorter than 5 or greater than 155
# Moved down since we're not evaluating any sessions with < 5 final results anyway
df.drop(df[~df.product_action.map(len).between(5, 155)].index, inplace=True)
df = df.reset_index()
del df['index']
df.shape

(3298, 2)

In [9]:
# Symbolization
# Based on practical session 1. Counter doesn't seem that useful but unsure about alternative
counts = Counter([item for session in df['product_action'] for item in session])
symbol_alpha = {action : idx for idx, action in enumerate(counts, 1)}
print(counts, '\nSymbol alphabet: ', symbol_alpha)

# Overwriting the product action column.
# Casting as a string (from tuple) is necessary for translating to dict in Oracle function
df['session'] = df['product_action'].map(lambda session: tuple([symbol_alpha[action] for action in session]))
df = df[['session', 'purchase']]
df.head()

Counter({'view': 50851, 'detail': 15086, 'remove': 6362, 'add': 2619}) 
Symbol alphabet:  {'view': 1, 'detail': 2, 'add': 3, 'remove': 4}


,session,purchase
0,"(1, 2, 1, 1, 1, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, ...",0
1,"(1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 2, 1, ...",0
2,"(1, 4, 1, 1, 1)",0
3,"(4, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 1, 2, 1, ...",0
4,"(3, 1, 4, 1, 1, 1, 1, 2, 3, 1, 2, 1, 1, 2, 1, ...",1


In [10]:
# Split set into training and validation set
from sklearn.model_selection import train_test_split

# No real reason for the selected ratio. In PS they used 70/30. Don't forget to set state.
training_set, eval_set = train_test_split(df, test_size=0.2, random_state=123, shuffle=False)
eval_set.head()

,session,purchase
2638,"(1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1)",0
2639,"(1, 1, 1, 1, 1, 4, 3, 1, 1, 1, 2, 1, 1, 1, 1, ...",1
2640,"(1, 1, 1, 1, 2, 1)",0
2641,"(4, 1, 2, 1, 4, 1, 2, 1, 1, 1, 2)",0
2642,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",0


In [15]:
from collections import defaultdict

# Return an oracle file for N clicks after add event (e.g. event 0)
def make_oracle(dataset, max_clicks):
    
    """
    Find the count of each unique session, both for purchase and no purchase sessions
    Function creates an oracle based on the clicks post add-to-cart, so filters based on session length
    Since we dropped all actions up to and including first add we can start from index 0
    """
    correction = 1
    
    buy = dataset.session[(dataset['purchase'] == 1) & \
                            (dataset['session'].str.len() >= max_clicks)].value_counts() + correction
    nobuy = dataset.session[(dataset['purchase'] == 0) & \
                            (dataset['session'].str.len() >= max_clicks)].value_counts() + correction
           
    oracle = dict.fromkeys(dataset.session[dataset['session'].str.len() >= max_clicks])

    """
    Are we even calculating precision here? Is the assumption that everything is flagged as a purchase session?
    
    Correcting all results by 1 means all results are bumped down up 0 to 0.33, which seems substantial
    Not correcting means we have a lot of certain no-purchase results, maybe that's good?
    Correcting by decimals is an option but that sounds like a silly solution?
    """
    
    print("Number of clicks: ", max_clicks, "\tPurchase sessions:  ", len(buy), \
          "\tNo purchase sessions:  ", len(nobuy), "\tTotal: ", len(nobuy) + len(buy))
    
    for key in oracle.keys():
        nb, nn = buy.get(key, default = correction), nobuy.get(key, default = correction)
        oracle[key] = nb / (nb + nn)
    return oracle

# As per the assignment, oracles are made for 5, 10 and 15 post add-to-cart clicks
ev_oracle5 = make_oracle(eval_set, 5)
#ev_oracle10 = make_oracle(eval_set, 10)
#ev_oracle15 = make_oracle(eval_set, 15)

tr_oracle5 = make_oracle(training_set, 5)

Number of clicks:  5 	Purchase sessions:   116 	No purchase sessions:   476 	Total:  592
Number of clicks:  5 	Purchase sessions:   503 	No purchase sessions:   1679 	Total:  2182


In [16]:
for key, value in ev_oracle5.items():
    if key in tr_oracle5:
        print(key[0:5], "\tTraining: ", "{:.2f}".format(round(tr_oracle5[key], 2)), \
              "\tEval: ", "{:.2f}".format(round(value, 2)), \
              "\tDiff: ", "{:.2f}".format(round(tr_oracle5[key] - value, 2)))

(1, 1, 1, 1, 2) 	Training:  0.17 	Eval:  0.33 	Diff:  -0.17
(1, 1, 1, 1, 1) 	Training:  0.45 	Eval:  0.38 	Diff:  0.08
(1, 1, 2, 1, 1) 	Training:  0.14 	Eval:  0.20 	Diff:  -0.06
(1, 1, 2, 1, 1) 	Training:  0.50 	Eval:  0.20 	Diff:  0.30
(1, 1, 1, 2, 1) 	Training:  0.40 	Eval:  0.50 	Diff:  -0.10
(1, 1, 2, 1, 2) 	Training:  0.14 	Eval:  0.25 	Diff:  -0.11
(1, 1, 1, 1, 1) 	Training:  0.21 	Eval:  0.67 	Diff:  -0.45
(3, 1, 1, 1, 1) 	Training:  0.67 	Eval:  0.50 	Diff:  0.17
(1, 1, 1, 1, 2) 	Training:  0.25 	Eval:  0.33 	Diff:  -0.08
(1, 1, 1, 1, 1) 	Training:  0.30 	Eval:  0.50 	Diff:  -0.20
(1, 1, 1, 1, 2) 	Training:  0.33 	Eval:  0.25 	Diff:  0.08
(1, 1, 1, 2, 1) 	Training:  0.20 	Eval:  0.67 	Diff:  -0.47
(2, 1, 1, 2, 1) 	Training:  0.20 	Eval:  0.33 	Diff:  -0.13
(1, 2, 1, 1, 1) 	Training:  0.20 	Eval:  0.33 	Diff:  -0.13
(2, 1, 2, 1, 1) 	Training:  0.33 	Eval:  0.33 	Diff:  0.00
(1, 2, 1, 1, 1) 	Training:  0.36 	Eval:  0.50 	Diff:  -0.14
(1, 2, 1, 1, 1) 	Training:  0.25 	Eval:  0.33